In [29]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

## REPORTE
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

In [30]:
# ruta de carpera donde estan las llaves
ruta = open('../../ruta.txt')
ruta = ruta.readlines()[0]

In [32]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [33]:
ruta

'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [34]:
def generar_reporte(i):
    id_reg = i
    
    # SELECCIONAR REGISTRO
    query = {'query': 'select * from dataset_318090 where id = {id_ca}'.format(id_ca=id_reg)}
    project = 'https://app.amigocloud.com/api/v1/projects/33032/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    if data['observaciones'] == None:
        data['observaciones']='Sin observaciones'
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    print('CAÑERO: ' + insp.canhero)
    #FOTOS
    query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_49781 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=insp.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/33032/sql'
    fotos = amigocloud.get(project, query_fotos)['data']
    
    #GENERAR DOCX
    doc = DocxTemplate(ruta + "/templates/tpl_rpt_operador-cosechadora.docx")
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta + '/fotos/' + foto['s3_filename'], width=Mm(60))})

    context = {'insp':insp, 'fotos':lista_fotos_inline}

    doc.render(context)

    # formato de nombre de archivo: "123_EVC_01-01-2022_NOMBRE"
    cod_nom = insp.canhero.split(' / ')
    file_name = ruta + '/_' + str(cod_nom[0]) + '_EVC_' + insp.fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.id)

    #doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
    doc.save(file_name + '.docx')

In [35]:
lista_id = list(range(152, 161))
lista_id

[152, 153, 154, 155, 156, 157, 158, 159, 160]

In [36]:
for i in lista_id:
    print('GENERANDO id = ' + str(i))
    generar_reporte(i)
print('END=====================================')

GENERANDO id = 152
CAÑERO: 446 / AGUILERA ANEZ JHONNY
GENERANDO id = 153
CAÑERO: 75 / AGUILERA TARADELLES MARIANO
GENERANDO id = 154
CAÑERO: 2899 / EGUEZ ELHAGE JOSE EDUARDO
GENERANDO id = 155
CAÑERO: 2899 / EGUEZ ELHAGE JOSE EDUARDO
GENERANDO id = 156
CAÑERO: 3378 / ESCOBAR MORATO VICTOR HUGO
GENERANDO id = 157
CAÑERO: 3378 / ESCOBAR MORATO VICTOR HUGO
GENERANDO id = 158
CAÑERO: 15001 / VARGAS PINTO RENNY EDWIN
GENERANDO id = 159
CAÑERO: 765 / AGROP. CATALA S.R.L.
GENERANDO id = 160
CAÑERO: 765 / AGROP. CATALA S.R.L.
END=====================================


## CONVERTIR A PDF

In [39]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 9 INFORMES
9 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/15001_EVC_09-05-2023_VARGAS PINTO RENNY EDWIN_158.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/2899_EVC_06-05-2023_EGUEZ ELHAGE JOSE EDUARDO_154.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/2899_EVC_06-05-2023_EGUEZ ELHAGE JOSE EDUARDO_155.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3378_EVC_09-05-2023_ESCOBAR MORATO VICTOR HUGO_156.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3378_EVC_09-05-2023_ESCOBAR MORATO VICTOR HUGO_157.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/446_EVC_05-05-2023_AGUILERA ANEZ JHONNY_152.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/75_EVC_05-05-2023_AGUILERA TARADELLES MARIANO_153.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/765_EVC_09-05-2023_AGROP. CATALA S.R.

In [43]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\UTEA\Downloads\\' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/15001_EVC_09-05-2023_VARGAS PINTO RENNY EDWIN_158.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/15001_EVC_09-05-2023_VARGAS PINTO RENNY EDWIN_158.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/15001_EVC_09-05-2023_VARGAS PINTO RENNY EDWIN_158.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 2 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2899_EVC_06-05-2023_EGUEZ ELHAGE JOSE EDUARDO_154.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2899_EVC_06-05-2023_EGUEZ ELHAGE JOSE EDUARDO_154.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/2899_EVC_06-05-2023_EGUEZ ELHAGE JOSE EDUARDO_154.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 3 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2899_EVC_06-05-2023_EGUEZ ELHAGE JOSE EDUARDO_155.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2899_EVC_06-05-2023_EGUEZ ELHAGE JOSE EDUARDO_155.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/2899_EVC_06-05-2023_EGUEZ ELHAGE JOSE EDUARDO_155.pdf


...

CONVIRTIENDO... INFORME 8 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/765_EVC_09-05-2023_AGROP. CATALA S.R.L._159.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/765_EVC_09-05-2023_AGROP. CATALA S.R.L._159.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/765_EVC_09-05-2023_AGROP. CATALA S.R.L._159.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 9 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/765_EVC_09-05-2023_AGROP. CATALA S.R.L._160.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/765_EVC_09-05-2023_AGROP. CATALA S.R.L._160.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/765_EVC_09-05-2023_AGROP. CATALA S.R.L._160.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END
